In [1]:
import pandas as pd
import numpy as np

# --- Configuration ---
DATA_PATH = 'data/'  # Update this if your files are in the same folder as your notebook

def load_and_merge_base_data(path):
    """
    Loads core F1 datasets and merges them into a main 'results' dataframe.
    Keeps granular data (laps, pits) separate to avoid row explosion.
    """
    # 1. Load the Core CSVs
    # Note: 'na_values' handles the specific way Ergast stores nulls
    print("Loading CSV files...")
    races = pd.read_csv(f'{path}races.csv', na_values='\\N')
    results = pd.read_csv(f'{path}results.csv', na_values='\\N')
    status = pd.read_csv(f'{path}status.csv', na_values='\\N')
    drivers = pd.read_csv(f'{path}drivers.csv', na_values='\\N')
    
    # Load Auxiliary Data (Keep separate for Feature Engineering later)
    # We don't merge these yet, but we load them to ensure they exist and look right
    lap_times = pd.read_csv(f'{path}lap_times.csv', na_values='\\N')
    pit_stops = pd.read_csv(f'{path}pit_stops.csv', na_values='\\N')

    # 2. Merge Logic (Creating the "Main" DataFrame)
    # Step A: Join Results with Race Info (Adds Year, Date, Circuit)
    # We rename 'name' to avoid collision (Race Name vs Driver Name)
    races_subset = races[['raceId', 'year', 'round', 'circuitId', 'name', 'date']]
    races_subset = races_subset.rename(columns={'name': 'race_name'})
    
    main_df = pd.merge(results, races_subset, on='raceId', how='left')

    # Step B: Join with Driver Info (Adds Name, Nationality)
    drivers_subset = drivers[['driverId', 'driverRef', 'code']]
    main_df = pd.merge(main_df, drivers_subset, on='driverId', how='left')

    # Step C: Join with Status (Adds "Collision", "Engine Failure" descriptions)
    main_df = pd.merge(main_df, status, on='statusId', how='left')

    print(f"Successfully loaded Main Dataframe with shape: {main_df.shape}")
    print(f"Auxiliary Data Loaded: Lap Times ({lap_times.shape[0]} rows), Pit Stops ({pit_stops.shape[0]} rows)")
    
    return main_df, lap_times, pit_stops

# Execution
raw_main_df, raw_laps_df, raw_pits_df = load_and_merge_base_data(DATA_PATH)

Loading CSV files...
Successfully loaded Main Dataframe with shape: (26759, 26)
Auxiliary Data Loaded: Lap Times (589081 rows), Pit Stops (11371 rows)


In [2]:
def clean_f1_data(df, laps_df, pits_df):
    """
    Cleans the merged dataframe and auxiliary frames:
    1. Filters for Modern Era (2000+)
    2. Converts time columns to numeric
    3. Handles missing values
    """
    print("Starting Data Cleaning...")
    
    # 1. Filter for Modern Era (2000-2023)
    # Metric consistency is better in the modern era
    df_clean = df[df['year'] >= 2000].copy()
    
    # Filter auxiliary tables to match the same raceIds (optimization)
    valid_races = df_clean['raceId'].unique()
    laps_clean = laps_df[laps_df['raceId'].isin(valid_races)].copy()
    pits_clean = pits_df[pits_df['raceId'].isin(valid_races)].copy()

    # 2. Fix Data Types in Main DataFrame
    # 'fastestLapTime' is often a string "1:30.000", we might need to drop or parse it.
    # For this project, we primarily need 'milliseconds' for the Tension Metric.
    
    # Convert 'milliseconds' to numeric, forcing errors to NaN (some DNFs have no time)
    df_clean['milliseconds'] = pd.to_numeric(df_clean['milliseconds'], errors='coerce')
    
    # Convert 'grid' and 'positionOrder' to integers
    df_clean['grid'] = df_clean['grid'].fillna(0).astype(int)
    df_clean['positionOrder'] = df_clean['positionOrder'].astype(int)

    # 3. Clean Auxiliary Data (Lap Times)
    # We need 'milliseconds' in laps for calculations
    laps_clean['milliseconds'] = pd.to_numeric(laps_clean['milliseconds'], errors='coerce')
    laps_clean['lap'] = laps_clean['lap'].astype(int)
    
    # 4. Handle "Did Not Start" / "Withdrawn"
    # We keep them for now as they contribute to the "Chaos" metric, 
    # but we ensure their finishing position isn't misleading.
    
    print("Cleaning Complete.")
    print(f"Cleaned Main Shape: {df_clean.shape}")
    print(f"Cleaned Laps Shape: {laps_clean.shape}")
    
    return df_clean, laps_clean, pits_clean

# Execution
final_df, final_laps, final_pits = clean_f1_data(raw_main_df, raw_laps_df, raw_pits_df)

# Sanity Check: Show the first 5 rows
final_df.head()

Starting Data Cleaning...
Cleaning Complete.
Cleaned Main Shape: (10079, 26)
Cleaned Laps Shape: (525465, 6)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLapSpeed,statusId,year,round,circuitId,race_name,date,driverRef,code,status
0,1,18,1,1,22.0,1,1.0,1,1,10.0,...,218.300,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,Finished
1,2,18,2,2,3.0,5,2.0,2,2,8.0,...,217.586,1,2008,1,1,Australian Grand Prix,2008-03-16,heidfeld,HEI,Finished
2,3,18,3,3,7.0,7,3.0,3,3,6.0,...,216.719,1,2008,1,1,Australian Grand Prix,2008-03-16,rosberg,ROS,Finished
3,4,18,4,4,5.0,11,4.0,4,4,5.0,...,215.464,1,2008,1,1,Australian Grand Prix,2008-03-16,alonso,ALO,Finished
4,5,18,5,1,23.0,3,5.0,5,5,4.0,...,218.385,1,2008,1,1,Australian Grand Prix,2008-03-16,kovalainen,KOV,Finished
